In [18]:
import substra
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from pathlib import Path


In [19]:
client = substra.Client.from_config_file("node_A")
cp_key = "be1cf90d-929f-4a74-bbd4-d212272f073c"
test_data_path = Path('/') / 'home' / 'user' / 'data' / 'test'


In [20]:
traintuples = client.list_traintuple(filters=[f'traintuple:compute_plan_key:{cp_key}'])


In [21]:
rank_max = 0
tt_key = ""

for tt in traintuples:
    if tt.rank > rank_max:
        rank_max = tt.rank
        tt_key = tt.key

In [22]:

# get the last traintuple of the compute plan
traintuple = client.get_traintuple(tt_key)

# Download the associated model
model_path = Path("") / 'out'
model_filename = f'model_{traintuple.train.models[0].key}'
client.download_model_from_traintuple(traintuple.key, folder=model_path)

# SPECIFIC TO YOUR ALGO
# -----------------------
#
# Here we are loading the model that your algo produced
# so if you change the algo, for example if you use PyTorch,
# then you might need to change this part as well.

# Load the model and create predictions: this code depends on the algo code
model = keras.models.load_model(str(model_path / model_filename))

image_size = (180, 180)
batch_size = 32
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_data_path,
    labels="inferred",
    label_mode="binary",
    shuffle=False,
    seed=0,
    image_size=image_size,
    batch_size=batch_size,
)

predictions = np.array([])
labels = np.array([])
for x, y in test_ds:
    predictions = np.concatenate([predictions, model.predict(x).ravel()])
    labels = np.concatenate([labels, y.numpy().ravel()])
file_paths = test_ds.file_paths

df_submission = pd.DataFrame(data={
    'file_paths': file_paths,
    'predictions': predictions
})
df_submission["file_paths"] = df_submission["file_paths"].apply(
    lambda x: x.replace("/home/user/data/test", "/data/challenges_data/test"))

submission_filepath = Path(
    __file__).resolve().parents[1] / 'out' / 'df_submission.csv'
df_submission.to_csv(submission_filepath, index=False)

print(df_submission.head())

print(f"Created the file at {submission_filepath}")

# Delete the downloaded model
(model_path / model_filename).unlink()

TypeError: 'NoneType' object is not subscriptable